In [2]:
import torch
from torch.utils.data import DataLoader

from models.linear_decoder import LinearDecoder
from bravo_datasets import CityscapesDataset
from inference import normal_eval
from clean_dataset import visualize_batch

/home/artemis/miniconda3/envs/benchmark-vfm-ss/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
encoder_name = ['vit_large_patch14_dinov2', "eva02_large_patch14_224", "depth-anything/Depth-Anything-V2-Base-hf"]
img_size = (1024, 1024) 
num_classes = 19
device = torch.device('cuda:0')
checkpoint_path_dinov2large = '/media/data/workspace_Giuseppe/code/checkpoints/CityScapes/dinov2largepatch8linear/best-checkpoint-epoch=41-val_0_miou=0.85.ckpt'
checkpoint_path_depthv2base = '/media/data/workspace_Giuseppe/code/checkpoints/CityScapes/depthv2base_16psize/best-checkpoint-epoch=36-val_0_miou=0.81.ckpt'

In [ ]:
out_of_context = '/media/data/workspace_Giuseppe/code/datasets/BRAVO/outofcontext'
ignore_idx_out_of_context = [0,1,9]
synrain = '/media/data/workspace_Giuseppe/code/datasets/BRAVO/synrain'

dataset = CityscapesDataset(root_dir=out_of_context, ignore_idx=ignore_idx_out_of_context)
eval_dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=8)

In [4]:
patch_size = 16
model = LinearDecoder(
    encoder_name[2],
    num_classes = num_classes,
    img_size = img_size,
    sub_norm = False, 
    patch_size = patch_size,
    pretrained = True
)

model.to(device)
model.eval()

checkpoint = torch.load(checkpoint_path_depthv2base)
  
model.load_state_dict(checkpoint['state_dict'])

/home/artemis/miniconda3/envs/benchmark-vfm-ss/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/artemis/miniconda3/envs/benchmark-vfm-ss/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


<All keys matched successfully>

In [ ]:
normal_eval(model=model, 
          img_size=img_size, 
          device=device, 
          num_classes=num_classes, 
          eval_dataloader=eval_dataloader)

----